In [1]:
!pip install ultralytics

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.listdir('/content/drive/MyDrive')

In [4]:
!pip install opencv-python

In [5]:
from ultralytics import YOLO
import torch
from google.colab import drive
import os

In [7]:
# Extract frames.zip from Google Drive
print("\n=== Extracting Dataset ===")
!rm -rf /content/frames
!unzip -o -q "/content/drive/MyDrive/frames.zip" -d "/content/"


=== Extracting Dataset ===


In [8]:
DATASET_PATH = "/content/frames"
DATA_YAML = os.path.join(DATASET_PATH, "data.yaml")

In [9]:
print("\n=== Verifying Dataset Structure ===")

# Check train/val/test folders
for split in ['train', 'val', 'test']:
    split_img_path = os.path.join(DATASET_PATH, split, 'images')
    split_lbl_path = os.path.join(DATASET_PATH, split, 'labels')

    if os.path.exists(split_img_path):
        img_files = [f for f in os.listdir(split_img_path)
                     if f.endswith(('.jpg', '.png', '.jpeg', '.JPG', '.PNG', '.JPEG'))]
        print(f"{split.upper()}: {len(img_files)} images", end='')

        if os.path.exists(split_lbl_path):
            lbl_files = [f for f in os.listdir(split_lbl_path) if f.endswith('.txt')]
            print(f", {len(lbl_files)} labels")
        else:
            print(f" - WARNING: labels folder not found!")
    else:
        print(f"{split.upper()}: folder not found!")



=== Verifying Dataset Structure ===
TRAIN: 819 images, 819 labels
VAL: 274 images, 274 labels
TEST: 153 images, 153 labels


In [10]:
print("\n=== Updating data.yaml with absolute path ===")

if os.path.exists(DATA_YAML):
    # Backup original
    with open(DATA_YAML, 'r') as f:
        original_content = f.read()
    with open(os.path.join(DATASET_PATH, "data_original.yaml"), 'w') as f:
        f.write(original_content)
    print("✅ Original data.yaml backed up as data_original.yaml")

# Create data.yaml with absolute path (required for YOLO)
data_yaml_content = f"""# YOLO Dataset Configuration File
# Gas Tank Detection Dataset

# Dataset paths (absolute path)
path: {DATASET_PATH}
train: train/images
val: val/images
test: test/images

# Classes
names:
  0: gas_bottle

# Number of classes
nc: 1

# Dataset information
dataset_name: "Gas Tank Detection Dataset"
description: "Dataset for detecting gas tanks from multiple camera angles"
camera_angles: [back_left, back_right, front, top]
created_by: "GASSY Project"
version: "1.0"
date_created: "2025-10-18"
"""

with open(DATA_YAML, 'w') as f:
    f.write(data_yaml_content)

print("✅ data.yaml updated with absolute path!")
print("\nUpdated data.yaml:")
print(data_yaml_content)

# Verify paths exist
print("\n=== Verifying paths ===")
for split in ['train', 'val', 'test']:
    full_path = os.path.join(DATASET_PATH, split, 'images')
    exists = os.path.exists(full_path)
    print(f"{split}: {full_path} - {'✓' if exists else '✗'}")

# ============================
# 5. CREATE/UPDATE DATA.YAML
# ============================

print("\n=== Creating data.yaml ===")

data_yaml_content = f"""# YOLO Dataset Configuration File
# Gas Tank Detection Dataset

# Dataset paths
path: {DATASET_PATH}
train: train/images
val: val/images
test: test/images

# Classes
names:
  0: gas_tank

# Number of classes
nc: 1
"""

with open(DATA_YAML, 'w') as f:
    f.write(data_yaml_content)

print("✅ data.yaml created successfully!")
print("\ndata.yaml content:")
print(data_yaml_content)


=== Updating data.yaml with absolute path ===
✅ Original data.yaml backed up as data_original.yaml
✅ data.yaml updated with absolute path!

Updated data.yaml:
# YOLO Dataset Configuration File
# Gas Tank Detection Dataset

# Dataset paths (absolute path)
path: /content/frames
train: train/images
val: val/images
test: test/images

# Classes
names:
  0: gas_bottle

# Number of classes
nc: 1

# Dataset information
dataset_name: "Gas Tank Detection Dataset"
description: "Dataset for detecting gas tanks from multiple camera angles"
camera_angles: [back_left, back_right, front, top]
created_by: "GASSY Project"
version: "1.0"
date_created: "2025-10-18"


=== Verifying paths ===
train: /content/frames/train/images - ✓
val: /content/frames/val/images - ✓
test: /content/frames/test/images - ✓

=== Creating data.yaml ===
✅ data.yaml created successfully!

data.yaml content:
# YOLO Dataset Configuration File
# Gas Tank Detection Dataset

# Dataset paths
path: /content/frames
train: train/images
v

In [11]:
print("\n=== Loading YOLOv11 Small Model ===")
model = YOLO('yolo11s.pt')

print("✅ Model loaded successfully!")

# Training parameters - Optimized for Tesla T4 (I'm using google colab)
EPOCHS = 100
BATCH_SIZE = 8 # the value 16 scared me so I chose 8
IMG_SIZE = 640  # Standard YOLO input size
PATIENCE = 50  # Early stopping patience
WORKERS = 8  # Number of dataloader workers for T4 (the gpu i'm using :))


=== Loading YOLOv11 Small Model ===
✅ Model loaded successfully!


In [12]:
print("\n=== Training Configuration (Tesla T4 Optimized) ===")
print(f"Epochs: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE} (optimized for T4 GPU)")
print(f"Image Size: {IMG_SIZE}")
print(f"Early Stopping Patience: {PATIENCE}")
print(f"Workers: {WORKERS}")


=== Training Configuration (Tesla T4 Optimized) ===
Epochs: 100
Batch Size: 8 (optimized for T4 GPU)
Image Size: 640
Early Stopping Patience: 50
Workers: 8


In [13]:
results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    patience=PATIENCE,
    workers=WORKERS,
    device=0,
    project='gas_tank_detection',
    name='yolo11s_run',
    exist_ok=True,
    single_cls=True,  # Important: single class detection
    amp=True,  # Good for T4 GPU
    seed=42  # For reproducibility
)


Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/frames/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11s_run, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=50, perspective=0.0, plots=True, pose=12.

In [14]:
print("\n=== Validating Model ===")
metrics = model.val()

print(f"\n📊 Validation Results:")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")



=== Validating Model ===
Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2460.9±760.2 MB/s, size: 112.8 KB)
val: Scanning /content/frames/val/labels.cache... 274 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 274/274 522.4Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 18/18 4.7it/s 3.9s
                   all        274        274          1          1      0.995       0.99
Speed: 2.0ms preprocess, 7.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/runs/detect/val

📊 Validation Results:
mAP50: 0.9950
mAP50-95: 0.9897
Precision: 1.0000
Recall: 1.0000


In [15]:
print("\n=== Testing on Test Set ===")
test_results = model.val(data=DATA_YAML, split='test')

print(f"\n📊 Test Results:")
print(f"mAP50: {test_results.box.map50:.4f}")
print(f"mAP50-95: {test_results.box.map:.4f}")


=== Testing on Test Set ===
Ultralytics 8.3.217 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1624.4±870.3 MB/s, size: 109.1 KB)
val: Scanning /content/frames/test/labels... 153 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 153/153 2.5Kit/s 0.1s
val: New cache created: /content/frames/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 10/10 3.4it/s 2.9s
                   all        153        153          1          1      0.995      0.988
Speed: 3.1ms preprocess, 8.0ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /content/runs/detect/val2

📊 Test Results:
mAP50: 0.9950
mAP50-95: 0.9884


In [16]:
# Already saved to Google Drive
print("Model location:")
print("- Best: /content/drive/MyDrive/gas_tank_detection/yolo11s_run/weights/best.pt")
print("- Last: /content/drive/MyDrive/gas_tank_detection/yolo11s_run/weights/last.pt")

Model location:
- Best: /content/drive/MyDrive/gas_tank_detection/yolo11s_run/weights/best.pt
- Last: /content/drive/MyDrive/gas_tank_detection/yolo11s_run/weights/last.pt


In [20]:
# Load best weights
best_model = YOLO('gas_tank_detection/yolo11s_run/weights/best.pt')
# Test on a few sample images from test set
test_img_path = os.path.join(DATASET_PATH, "test", "images")
if os.path.exists(test_img_path):
    test_files = [os.path.join(test_img_path, f) for f in os.listdir(test_img_path)
                  if f.endswith(('.jpg', '.png', '.jpeg', '.JPG', '.PNG', '.JPEG'))][:5]

    if test_files:
        print(f"Running inference on {len(test_files)} test images...")
        results = best_model(test_files)

        # Save predictions
        for i, r in enumerate(results):
            r.save(filename=f'test_prediction_{i}.jpg')

        print(f"Predictions saved: test_prediction_0.jpg to test_prediction_{len(test_files)-1}.jpg")


Running inference on 5 test images...

0: 480x640 1 item, 9.6ms
1: 480x640 1 item, 9.6ms
2: 480x640 1 item, 9.6ms
3: 480x640 1 item, 9.6ms
4: 480x640 1 item, 9.6ms
Speed: 1.3ms preprocess, 9.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
Predictions saved: test_prediction_0.jpg to test_prediction_4.jpg


In [19]:
print("\n=== Saving Results to Google Drive ===")
!cp -r gas_tank_detection /content/drive/MyDrive/


=== Saving Results to Google Drive ===
